# Using THREDDS Server to retrive a NcML file with XArray


The THREDDS data storing netCDF file on PAVICS has some public and private directories. Data from public directories can be accessed anonymously, while data from private directories can require authentication, depending on rules applied on certain directories. This notebook shows how to access public NcML files with `xarray`, stored under the THREDDS server.

The PAVICS THREDDS server has two test access points to validate the use of direct THREDDS requests and protected ones through Twitcher proxied requests.anonymouslyanonymously

In [16]:
# define some useful variables for following steps
import os

PAVICS_HOST = os.getenv("PAVICS_HOST", "pavics.ouranos.ca")
PROTECTED_THREDDS_URL = f"https://{PAVICS_HOST}/twitcher/ows/proxy/thredds"
TEST_PUBLIC_THREDDS_URL = f"https://{PAVICS_HOST}/testthredds"

assert PAVICS_HOST != "", "Invalid PAVICS HOST value."
print("PROTECTED THREDDS URL:", PROTECTED_THREDDS_URL)
print("TEST PUBLIC THREDDS URL:", TEST_PUBLIC_THREDDS_URL)

PROTECTED THREDDS URL: https://pavics.ouranos.ca/twitcher/ows/proxy/thredds
TEST PUBLIC THREDDS URL: https://pavics.ouranos.ca/testthredds


First let's just open an unsecured link.

In [17]:
# NBVAL_IGNORE_OUTPUT

from urllib.parse import quote

import xarray as xr

PUBLIC_NCML_PATH = "dodsC/testdatasets/CanDCS-U6/day_BCCAQv2+ANUSPLIN300_MPI-ESM1-2-LR_historical+ssp245_r1i1p1f1_gn_1950-2100.ncml"
PUBLIC_NCML_PATH = quote(PUBLIC_NCML_PATH)

PUBLIC_NCML_URL = f"{TEST_PUBLIC_THREDDS_URL}/{PUBLIC_NCML_PATH}"
PUBLIC_NCML_URL

'https://pavics.ouranos.ca/testthredds/dodsC/testdatasets/CanDCS-U6/day_BCCAQv2%2BANUSPLIN300_MPI-ESM1-2-LR_historical%2Bssp245_r1i1p1f1_gn_1950-2100.ncml'

In [18]:
display(xr.open_dataset(PUBLIC_NCML_URL))

<xarray.Dataset>
Dimensions:  (lon: 1068, lat: 510, time: 55152)
Coordinates:
  * lon      (lon) float64 -141.0 -140.9 -140.8 -140.7 ... -52.21 -52.13 -52.04
  * lat      (lat) float64 41.04 41.12 41.21 41.29 ... 83.21 83.29 83.37 83.46
  * time     (time) datetime64[ns] 1950-01-01 1950-01-02 ... 2100-12-31
Data variables:
    tasmin   (time, lat, lon) float32 ...
    tasmax   (time, lat, lon) float32 ...
    pr       (time, lat, lon) float32 ...
Attributes: (12/54)
    institution:                Pacific Climate Impacts Consortium (PCIC), Vi...
    contact:                    Pacific Climate Impacts Consortium
    Conventions:                CF-1.7 CMIP-6.2
    domain:                     Canada
    creation_date:              2021-02-17T15:52:04PST
    frequency:                  day
    ...                         ...
    bias_adjustment_method_id:  BCCAQv2
    bias_adjustment_reference:  Alex J. Cannon, Stephen R. Sobie, and Trevor ...
    license_type:               permissive
    license:                    https://open.canada.ca/en/open-government-lic...
    terms_of_use:               Further terms of use at https://pacificclimat...
    attribution:                Citation information for this dataset can be ...

Now let's do the same with a secured link.

Note that the link is expected to be publicly accessible, but protected behind Twitcher proxy.

We use the same file stored under the protected location, so the same result should be the same as previous.

In [20]:
PROTECTED_NCML_URL = f"{PROTECTED_THREDDS_URL}/{PUBLIC_NCML_PATH}"
PROTECTED_NCML_URL

'https://pavics.ouranos.ca/twitcher/ows/proxy/thredds/dodsC/testdatasets/CanDCS-U6/day_BCCAQv2%2BANUSPLIN300_MPI-ESM1-2-LR_historical%2Bssp245_r1i1p1f1_gn_1950-2100.ncml'

In [22]:
display(xr.open_dataset(PROTECTED_NCML_URL))

syntax error, unexpected $end, expecting SCAN_ATTR or SCAN_DATASET or SCAN_ERROR
context: ^


OSError: [Errno -72] NetCDF: Malformed or inaccessible DAP2 DDS or DAP4 DMR response: 'https://pavics.ouranos.ca/twitcher/ows/proxy/thredds/dodsC/testdatasets/CanDCS-U6/day_BCCAQv2%2BANUSPLIN300_MPI-ESM1-2-LR_historical%2Bssp245_r1i1p1f1_gn_1950-2100.ncml'